In [19]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

# First part of the homework #
## find the top news story headline and the summary ##

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.mars_db
collection = db.articles

In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'lxml')

In [5]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='slide')

# loop through results to get items
for result in results:

    title = result.find('div', class_='content_title').text

    paragraph = result.find('div', class_='rollover_description_inner').text

    date = result.find('div', class_='list_date')


    print('-----------------')
    print(title)
    print(paragraph)
    print(date) #date isn't found.  on webpage there is a div but not in response

-----------------


NASA Ingenuity Mars Helicopter Prepares for First Flight



Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  

None
-----------------


NASA's Perseverance Drives on Mars' Terrain for First Time



The first trek of the agency’s largest, most advanced rover yet on the Red Planet marks a major milestone before science operations get under way.

None
-----------------


NASA Awards Mars Ascent Propulsion System Contract for Sample Return



The award moves NASA and ESA a step closer to realizing Mars Sample Return, an ambitious planetary exploration program that will build upon decades of science, knowledge, and experience.

None
-----------------


NASA to Provide Update on Perseverance ‘Firsts' Since Mars Landing 



Mission team members are participating in a virtual teleconference to discuss milestones achieved so far since the Feb. 18 landing and those to come.

None
-----------------


NASA's Mars Perse

In [20]:
# assign response to variables and print

news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='rollover_description_inner').text

print('----------')
print(news_title)
print(news_p)

----------


NASA Ingenuity Mars Helicopter Prepares for First Flight



Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  



In [18]:
print(news_title)
print(news_p)



NASA Ingenuity Mars Helicopter Prepares for First Flight



Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  



 # Second part of homework (get the featured image url from JPL page)#

In [21]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - There is no [mac64] chromedriver for browser 89.0.4389 in cache
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/Howard/.wdm/drivers/chromedriver/mac64/89.0.4389.23]


In [22]:
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

In [36]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# find the parent div of the image
img_url = soup.find('div', class_='floating_text_area')
# find the href within the div
ref = img_url.find('a')['href']
# build the final link for the featured image
featured_image_url = ['https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + ref]

In [38]:
print(featured_image_url)

['https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg']


In [43]:
browser.quit()

# Third part of homework (use pandas to get table from webpage)#

In [39]:
url = 'https://space-facts.com/mars/'

In [49]:
import pandas as pd
tables = pd.read_html(url)
table = tables[0]
table

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [47]:
table.to_html('table.html')

In [48]:
!open table.html